# Классификация интентов
В этом ноутбуке произведем классификацию интентов

Скачаем данные и установим необходимое окружение:

In [1]:
! wget https://huggingface.co/datasets/blinoff/medical_qa_ru_data/resolve/main/medical_qa_ru_data.csv

--2024-02-10 16:06:31--  https://huggingface.co/datasets/blinoff/medical_qa_ru_data/resolve/main/medical_qa_ru_data.csv
Resolving huggingface.co (huggingface.co)... 18.164.174.17, 18.164.174.23, 18.164.174.118, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.17|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/datasets/blinoff/medical_qa_ru_data/6aca4fb6dae499745c9ca1dfad8a0ade809573daf228c923fdf846ba4a639b50?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27medical_qa_ru_data.csv%3B+filename%3D%22medical_qa_ru_data.csv%22%3B&response-content-type=text%2Fcsv&Expires=1707840391&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNzg0MDM5MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9kYXRhc2V0cy9ibGlub2ZmL21lZGljYWxfcWFfcnVfZGF0YS82YWNhNGZiNmRhZTQ5OTc0NWM5Y2ExZGZhZDhhMGFkZTgwOTU3M2RhZjIyOGM5MjNmZGY4NDZiYTRhNjM5YjUwP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9z

In [2]:
!pip install -q transformers

In [3]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm_notebook
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, classification_report

Примонтируем гугл-диск и скачаем файл с вопросами и ответами мейла:

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
!ls "/content/drive/My Drive/NLP/Course_Project/"

Chat-bot.ipynb	Classifier_of_intent.ipynb  Otvety.txt	QA_medical_generator.ipynb  questions.txt


In [6]:
path =  "/content/drive/My Drive/NLP/Course_Project/"

Так как этот файл мы используем лишь для классификации интентов, сохраним только вопросы:

In [7]:
assert True

#Small preprocess of the answers

written = False

with open(path + "questions.txt", "w") as fout:
    with open(path + "Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written:
                fout.write(line.replace("\t", " ").strip() + "\n")
                written = True
                continue

<ipython-input-7-18c73794ee56>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

Обернем 1-й интент в датасет и назначим ему класс 0:

In [8]:
data_1 = {'id': [], 'text': [], 'class': []}
with open(path + "questions.txt", "r") as f:
    for idx, line in enumerate(f):
      data_1['id'].append(idx)
      data_1['text'].append(line.strip())
      data_1['class'].append(0)
data_1 = pd.DataFrame(data_1)

In [9]:
data_1

,id,text,class
0,0,,0
1,1,вопрос о ТДВ)) давно и хорошо отдыхаем)) ЛИЧНО...,0
2,2,Как парни относятся к цветным линзам? Если у д...,0
3,3,"Что делать, сегодня нашёл 2 миллиона рублей? .",0
4,4,Эбу в двенашке называется Итэлма что за эбу? .,0
...,...,...,...
1163417,1163417,Между словами ПРЕЗИДЕНТ и РЕЗИДЕНТ есть что ли...,0
1163418,1163418,"""Если это мое, то оно никуда от меня не денетс...",0
1163419,1163419,А Вы халяву любите или совесть имеете???) .,0
1163420,1163420,Так много разных гороскопов кто-нибудь может п...,0


Посмотрим датасет для второго интента:



In [10]:
data_2 = pd.read_csv('medical_qa_ru_data.csv')
data_2

,date,categ,theme,desc,ans,spec10
0,"8 Октября 2017, 11:55",Оториноларингология,Применение Ларипронта.,"Ларипронт 20 талеток,через каждые 2-3 часа.Оче...",Что вы им лечите? Длительность приема Ларипрон...,Отоларинголог
1,"20 Февраля 2019, 13:24",Акушерство,Беременность,"Здравствуйте, я на 7-8 неделе беременности. С ...","Здравствуйте, это может быть признаком раннего...",NaN
2,"17 Марта 2015, 18:31",Другое,гинекология,Здравствуйте месячные должны придти 23 марта в...,Выполните исследование хгч,NaN
3,"13 Января 2019, 19:38",Терапия,Занятия спорта после сдачи крови,"Завтра иду с утра сдавать кровь ТТГ, Т4СВ, Кал...","Можно.;\nЗдравствуйте , да, попейте сладкого ч...",Терапевт
4,"28 Ноября 2017, 21:58",Другое,Таблетки,Мне прописали пить Аллохол. Врач написала пить...,Препарат принимается после еды. Уточните это ...,NaN
...,...,...,...,...,...,...
190330,"31 Июля 2014, 11:31",Офтальмология,Отлетела в глаз пластмасса,"Здравствуйте! В глаз отлетел кусок пластмассы,...","Ромашка есть дома, альбуцид;\nМожно промыть лю...",Офтальмолог
190331,"16 Марта 2014, 21:52",Другое,здравствуйте скажите пожалуста где я могу посм...,NaN,"В ""консультанте""",NaN
190332,"22 Ноября 2018, 11:02",Общая хирургия,палец,"Здравствуйте, три дня назад порезала палец бу...","Здравствуйте , пока обработать Хлоргексидином ...",Хирург
190333,"2 Ноября 2018, 21:19",Эндокринология,Эндокринолог прописал пить сиофор 500 при саха...,NaN,Такие препараты принимаются постоянно;\nЗдравс...,NaN


Из ответов оставим только первый ответ. Сообщения разделены ';\n'

In [ ]:
# data_2.ans = data_2.ans.apply(lambda x: re.split(r';\n', str(x))[0])
# data_2.ans[0]

'Что вы им лечите? Длительность приема Ларипронта индивидуальна и устанавливается лечащим врачом.'

Из датасета оставим только столбцы 'desc' и 'ans', удалим строки с пустыми значениями, и назначим данному интенту класс 1:

In [11]:
data_2 = data_2[['desc', 'ans']]
data_2.dropna(subset=['desc', 'ans'], inplace=True)
data_2['class'] = 1
data_2

<ipython-input-11-8b2c5c659da8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2.dropna(subset=['desc', 'ans'], inplace=True)
<ipython-input-11-8b2c5c659da8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2['class'] = 1


,desc,ans,class
0,"Ларипронт 20 талеток,через каждые 2-3 часа.Оче...",Что вы им лечите? Длительность приема Ларипрон...,1
1,"Здравствуйте, я на 7-8 неделе беременности. С ...","Здравствуйте, это может быть признаком раннего...",1
2,Здравствуйте месячные должны придти 23 марта в...,Выполните исследование хгч,1
3,"Завтра иду с утра сдавать кровь ТТГ, Т4СВ, Кал...","Можно.;\nЗдравствуйте , да, попейте сладкого ч...",1
4,Мне прописали пить Аллохол. Врач написала пить...,Препарат принимается после еды. Уточните это ...,1
...,...,...,...
190328,"Не курю и не пью. Веду здоровый образ жизни, м...",В первую очередь вам необходимо обследовать пе...,1
190329,"Здравствуйте, очень прошу консультации. В лево...",Начинать следует с терапевтических методов. Ес...,1
190330,"Здравствуйте! В глаз отлетел кусок пластмассы,...","Ромашка есть дома, альбуцид;\nМожно промыть лю...",1
190332,"Здравствуйте, три дня назад порезала палец бу...","Здравствуйте , пока обработать Хлоргексидином ...",1


In [12]:
data_1 = data_1.sample(10000)
data_1

,id,text,class
953521,953521,"Русский актер, блистательно воплотивший образы...",0
259288,259288,"Помогите,пожалуйста, определить размер и стопу...",0
309288,309288,как решите номер 351 помогите пожалуйста напис...,0
629860,629860,Так и хоцца дать ему в глаз !! Ну не козёл ли?...,0
84629,84629,хочу со всеми Вами встретить новый год но не с...,0
...,...,...,...
534684,534684,Титаник сильный фильм о катастрофе для всех по...,0
496617,496617,Италия была когда - нибудь мировым чемпионом в...,0
938476,938476,"Как вы думаете, какую могла бы иметь профессию...",0
609720,609720,24. Ниже приведен перечень территориальных обр...,0


In [13]:
data_1 = data_1[['text', 'class']]

In [14]:
data_2 = data_2.sample(10000)

In [15]:
data_2

,desc,ans,class
36053,"Подскажите,пожалуйста, где можно в Москве пров...","Задайте этот вопрос в разделе ""клиники"";\nГоро...",1
98121,"Доброго времени суток,ребенку 9 лет,прошли экг...",По МКБ 10 - это другие неточно обозначенные б...,1
37421,Подсккжите голова раскалывается тяжесть где пе...,"У вас возможен ШОХ,покажитесь неврологу.;\nНач...",1
131836,"Здравствуйте,ситуация следующая,половой член п...",У вас может быть гарднереллез.). Помочь в вопр...,1
106228,Возможно ли такое: на сроке 8 недель узист ска...,"Да,так может быть.;\nПользователь поблагодарил...",1
...,...,...,...
103396,"Подскажите, что делать и какие прогнозы?\n ...","Здравствуйте, лечение вам стоило начинать рань...",1
11588,"Здравствуйте. Меня укусила пчела в четверг, в ...","Здравствуйте, продолжайте принимать антигистам...",1
83616,Здравствуйте. Может ли при грудном остеохондро...,Наиболее сильно может болеть желудок при шейно...,1
76606,Здравствуйте. Вопрос такой: после прививки пре...,Здравствуйте. Это не от прививки однозначно. Р...,1


In [16]:
data_2_tmp = data_2[['desc', 'class']]
data_2_tmp.rename(columns={'desc': 'text'}, inplace=True)
data_2_tmp

<ipython-input-16-6b1cbdea0a7a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2_tmp.rename(columns={'desc': 'text'}, inplace=True)


,text,class
36053,"Подскажите,пожалуйста, где можно в Москве пров...",1
98121,"Доброго времени суток,ребенку 9 лет,прошли экг...",1
37421,Подсккжите голова раскалывается тяжесть где пе...,1
131836,"Здравствуйте,ситуация следующая,половой член п...",1
106228,Возможно ли такое: на сроке 8 недель узист ска...,1
...,...,...
103396,"Подскажите, что делать и какие прогнозы?\n ...",1
11588,"Здравствуйте. Меня укусила пчела в четверг, в ...",1
83616,Здравствуйте. Может ли при грудном остеохондро...,1
76606,Здравствуйте. Вопрос такой: после прививки пре...,1


In [17]:
data = pd.concat([data_1, data_2_tmp], ignore_index=True)
data

,text,class
0,"Русский актер, блистательно воплотивший образы...",0
1,"Помогите,пожалуйста, определить размер и стопу...",0
2,как решите номер 351 помогите пожалуйста напис...,0
3,Так и хоцца дать ему в глаз !! Ну не козёл ли?...,0
4,хочу со всеми Вами встретить новый год но не с...,0
...,...,...
19995,"Подскажите, что делать и какие прогнозы?\n ...",1
19996,"Здравствуйте. Меня укусила пчела в четверг, в ...",1
19997,Здравствуйте. Может ли при грудном остеохондро...,1
19998,Здравствуйте. Вопрос такой: после прививки пре...,1


In [18]:
data = data.sample(frac=1).reset_index(drop=True)
data

,text,class
0,Почему нельзя пить детролекс ? В 22 года ? Есл...,1
1,Можно ли заказать железо за границей? Возможно...,0
2,Вы хотите меня избавить от комплексов? Сомнева...,0
3,я забил пароль и не могу вспомнить как можна е...,0
4,"Здравствуйте,была замершая беременность 12 нед...",1
...,...,...
19995,Здравствуйте помогите расшифровать анализы на ...,1
19996,Может ли один человек разрушить огромную стран...,0
19997,Можно ли отнести L'Homme Yves Saint Laurent к ...,0
19998,Решите кросворд пожалуйста срочно надо. 1.Свед...,0


Сохраним датасеты для последующей работы:

In [19]:
path =  "/content/drive/My Drive/NLP/Course_Project/"
data.to_csv(path + 'all_data.csv', encoding='utf-8', index=False)
data_2.to_csv(path + 'data_medical.csv', encoding='utf-8', index=False)

In [20]:
data = pd.read_csv(path+'all_data.csv')
data

,text,class
0,Почему нельзя пить детролекс ? В 22 года ? Есл...,1
1,Можно ли заказать железо за границей? Возможно...,0
2,Вы хотите меня избавить от комплексов? Сомнева...,0
3,я забил пароль и не могу вспомнить как можна е...,0
4,"Здравствуйте,была замершая беременность 12 нед...",1
...,...,...
19995,Здравствуйте помогите расшифровать анализы на ...,1
19996,Может ли один человек разрушить огромную стран...,0
19997,Можно ли отнести L'Homme Yves Saint Laurent к ...,0
19998,Решите кросворд пожалуйста срочно надо. 1.Свед...,0


Разобьем датасет на тренировочную и валидационную выборку:

In [21]:
text_train, text_valid, y_train, y_valid = train_test_split(data['text'],
                                                            data['class'],
                                                            test_size=.2,
                                                            shuffle=True,
                                                            stratify=data['class'],
                                                            random_state=42)

Загрузим мультилингвальную модель для классификации с количеством классов, равным 2:

In [22]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Токенизируем датасет и создадим датасет для обучения и валидации:

In [23]:
tf_train = tokenizer(text_train.tolist(), padding="max_length", truncation=True, return_tensors='tf')

In [24]:
tf_valid = tokenizer(text_valid.tolist(), padding="max_length", truncation=True, return_tensors='tf')

In [25]:
num_classes = 2
tf_y_train = tf.keras.utils.to_categorical(y_train.tolist(), num_classes)
tf_y_valid = tf.keras.utils.to_categorical(y_valid.tolist(), num_classes)

In [26]:
train_features = {x: tf_train[x] for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_y_train))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train)).batch(4)

valid_features = {x: tf_valid[x] for x in tokenizer.model_input_names}
valid_tf_dataset = tf.data.Dataset.from_tensor_slices((valid_features, tf_y_valid))
valid_tf_dataset = valid_tf_dataset.batch(4)

Заморозим бертовый слой, оставив для обучения только слой классификации, и скомпилируем модель:

In [27]:
model.layers[0].trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[tf.metrics.CategoricalAccuracy('accuracy')],
)

model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  177853440 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 177854978 (678.46 MB)
Trainable params: 1538 (6.01 KB)
Non-trainable params: 177853440 (678.46 MB)
_________________________________________________________________


Обучим модель:

In [28]:
model.fit(train_tf_dataset, validation_data=valid_tf_dataset, epochs=2)

Epoch 1/2
4000/4000 [==============================] - 985s 242ms/step - loss: 0.2800 - accuracy: 0.9034 - val_loss: 0.1507 - val_accuracy: 0.9470
Epoch 2/2
4000/4000 [==============================] - 996s 249ms/step - loss: 0.1860 - accuracy: 0.9325 - val_loss: 0.1286 - val_accuracy: 0.9510


Сохраним модель и токенайзер:

In [29]:
tokenizer.save_pretrained(path + 'tokenizer_intent_classifier/')
model.save_pretrained(path + 'model_intent_classifier/')

Функция обратного вызова для построения отчета классификации при валидации:

In [30]:
class Metrics(Callback):


    def on_test_end(self, epoch, logs=None):
        val_predict = model.predict(valid_tf_dataset).logits.argmax(axis=1)
        print('\n', classification_report(y_valid.astype('int32'), val_predict))

Построим отчет классификации на валидационном датасете:

In [31]:
model.evaluate(valid_tf_dataset, callbacks=[Metrics()])

1000/1000 [==============================] - 179s 176ms/step

               precision    recall  f1-score   support

           0       0.96      0.94      0.95      2000
           1       0.94      0.96      0.95      2000

    accuracy                           0.95      4000
   macro avg       0.95      0.95      0.95      4000
weighted avg       0.95      0.95      0.95      4000

1000/1000 [==============================] - 384s 384ms/step - loss: 0.1286 - accuracy: 0.9510


[0.1286180168390274, 0.9509999752044678]

In [32]:
tokenizer1 = AutoTokenizer.from_pretrained(path + 'tokenizer_intent_classifier/')
model1 = TFAutoModelForSequenceClassification.from_pretrained(path + 'model_intent_classifier/')

Some layers from the model checkpoint at /content/drive/My Drive/NLP/Course_Project/model_intent_classifier/ were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/My Drive/NLP/Course_Project/model_intent_classifier/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions w

In [33]:
def intent_classifier(sentence):
    sent_token = tokenizer1(sentence, padding="max_length", truncation=True, return_tensors='tf')
    predict = model1(sent_token).logits.numpy()
    print(predict)
    return predict.argmax()

In [35]:
intent_classifier('Привет!)')

[[ 2.114134  -2.0099614]]


0

In [38]:
intent_classifier('Что мне делать, если болит живот?')

[[-1.3962317  1.5068386]]


1

In [39]:
intent_classifier('Как ты?')

[[ 0.2388131  -0.19173963]]


0

In [40]:
intent_classifier('Не перестаёт болеть голова, какую таблетку принять?')

[[-2.2088995  2.2944434]]


1